In [26]:
import cv2
import numpy as np
import glob  # for read file from folder
import time  # for estimate processing time(optional)
import matplotlib.pyplot as plt
import os

In [33]:
SCALE_TO_HEIGHT = 60.0

def preprocess(image):
    scale_factor = SCALE_TO_HEIGHT / image.shape[0]
    width  = int(image.shape[1] * scale_factor)
    height = int(image.shape[0] * scale_factor)
    image = cv2.resize(image, (width, height), interpolation=cv2.INTER_AREA)
    image = cv2.medianBlur(image, 5)
    _, image = cv2.threshold(image,0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return image, scale_factor
    

def plot_gray(image):
    %matplotlib qt
    plt.figure(figsize=(20,15))
    return plt.imshow(image, cmap='Greys_r')

def find_features(main_image):
    correct_matches_dct = {}
    directory = 'samples/'
    for sample_image_name in os.listdir(directory):
        sample_image = cv2.imread(directory+sample_image_name, 0)
        sample_image, sample_scale_factor = preprocess(sample_image)
        orb = cv2.ORB_create()
        kp1, des1 = orb.detectAndCompute(main_image, None)
        kp2, des2 = orb.detectAndCompute(sample_image, None)
        bf = cv2.BFMatcher()
        matches = bf.knnMatch(des1, des2, k=2)
        correct_matches = []
        for m, n in matches:
            if m.distance < 0.75*n.distance:
                correct_matches.append([m])
                correct_matches_dct[sample_image_name.split('.')[0]]= len(correct_matches)
    correct_matches_dct = dict(sorted(correct_matches_dct.items(),
                                     key=lambda item: item[1],
                                     reverse=True))
    result = list(correct_matches_dct.keys())[0]
    print(result)
    return result

In [34]:
main_image = cv2.imread('template5.jpg', 0)
main_image_processed, main_image_scale_factor = preprocess(main_image)

In [35]:
features = find_features(main_image_processed)

IndexError: list index out of range

In [36]:
img = cv2.imread('image.jpg',0)
# Initiate ORB detector
orb = cv2.ORB_create()
# find the keypoints with ORB
kp = orb.detect(img,None)
# compute the descriptors with ORB
kp, des = orb.compute(img, kp)
print(len(kp), len(des))
# draw only keypoints location,not size and orientation
img2 = cv2.drawKeypoints(img, kp, None, color=(0,255,0), flags=0)
%matplotlib qt
plt.imshow(img2), plt.show()

500 500


(<matplotlib.image.AxesImage at 0x7f3259860490>, None)

In [ ]:
main_image = cv2.imread('template5.jpg')
gray_main_image, _ = preprocess(main_image)
contours = find_contours_of_cards(gray_main_image)
cards_location = find_coordinates_of_cards(contours, gray_main_image)
draw_rectangle_aroud_cards(cards_location, main_image)